In [1]:
import os

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import losses
from tensorflow.keras.layers import (Input, concatenate, Conv2D, MaxPooling2D,
                                     Conv2DTranspose, BatchNormalization, Dropout)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
def dice_coeff(y_true, y_pred):
    # Computes the dice coefficient between labels and predictions.
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    # Computes the dice loss between labels and predictions.
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def IoU(y_true, y_pred):
    # Computes the Intersection Over Union between labels and predictions.
    smooth=1.
    #flatten label and prediction tensors
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    total = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
    union = total - intersection
    IoU = (intersection + smooth) / (union + smooth)

    return IoU

def IoU_Loss(y_true, y_pred):
    # Computes the IoU loss between labels and predictions.
    return 1 - IoU(y_true, y_pred)

In [3]:
def unet_model(rows, cols, n_band, droprate=0.1):
    inputs = Input((rows, cols, n_band))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Dropout(droprate)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    pool1 = BatchNormalization()(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(droprate)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    pool2 = BatchNormalization()(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3= Dropout(droprate)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    pool3 = BatchNormalization()(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Dropout(droprate)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = BatchNormalization()(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Dropout(droprate)(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    up6 = BatchNormalization()(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Dropout(droprate)(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    up7 = BatchNormalization()(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Dropout(droprate)(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    up8 = BatchNormalization()(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Dropout(droprate)(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    up9 = BatchNormalization()(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Dropout(droprate)(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    model.compile(optimizer=Adam(learning_rate=1e-5), loss = losses.binary_crossentropy, metrics =['binary_accuracy', 'Precision', 'Recall'] )

    return model

In [ ]:
train_x = np.load('./image1.npy')
train_y = np.load('./label1.npy')
print(f"Shape of the traing sample: {train_x.shape}")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.25, random_state=42)
print(f"Training data shape: {x_train.shape} & Testing data shape: {x_test.shape}")

In [ ]:
unet = unet_model(128,128,4)
unet.summary()

In [ ]:
model_dir = os.path.join('./models','UNet.h5')
log_dir = os.path.join('./logs','UNet_log')
callback = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor="val_loss"),
    tf.keras.callbacks.ModelCheckpoint(
        model_dir,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only = False
    ),
    tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]
unet.fit(x_train, y_train, validation_split=0.1, batch_size=32, epochs=30, callbacks= callback)